In [1]:
import os
from pathlib import Path
from allensdk.core.brain_observatory_cache import BrainObservatoryCache

class AllenAPI:
    """Singleton class for accessing Allen Brain Observatory API"""
    
    _instance = None
    _boc = None  # Lazy-loaded BrainObservatoryCache instance

    def __new__(cls):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        return cls._instance

    @property
    def boc(self):
        """Lazy-load BrainObservatoryCache only when accessed."""
        if self._boc is None:
            allen_cache_path = os.environ.get('CAIM_ALLEN_CACHE_PATH')
            if not allen_cache_path:
                raise ValueError("AllenAPI requires a valid cache path. Set `CAIM_ALLEN_CACHE_PATH` in .env.")

            manifest_path = Path(allen_cache_path) / 'brain_observatory_manifest.json'
            self._boc = BrainObservatoryCache(manifest_file=str(manifest_path))

        return self._boc

    def get_boc(self):
        """Retrieve the BrainObservatoryCache object, ensuring it is initialized."""
        return self.boc

# Create a global instance so that all files can use it
allen_api = AllenAPI()

class AllenStimuliFetchStep(PipelineStep):
    """
    Fetches data from the Allen Brain Observatory.
    The session IDs are hard-coded since the stimuli are always the same.
    """
    # Hard-coded sessions
    SESSION_A = 501704220
    SESSION_B = 501559087
    SESSION_C = 501474098

    def __init__(self, boc):
        """
        :param boc: BrainObservatoryCache instance (via the AllenAPI singleton).
        """
        self.boc = boc

    def process(self, data):
        """
        Expects data to be either None or have (container_id, session, stimulus).
        We fetch a dictionary of raw stimuli arrays, store them in data['raw_data_dct'].
        """
        if isinstance(data, tuple):
            container_id, session, stimulus = data
            data = {'container_id': container_id, 'session': session, 'stimulus': stimulus}
        elif data is None:
            data = {}

        raw_data_dct = {}

        movie_one_dataset = self.boc.get_ophys_experiment_data(self.SESSION_A)
        raw_data_dct['natural_movie_one'] = movie_one_dataset.get_stimulus_template('natural_movie_one')

        movie_two_dataset = self.boc.get_ophys_experiment_data(self.SESSION_C)
        raw_data_dct['natural_movie_two'] = movie_two_dataset.get_stimulus_template('natural_movie_two')

        movie_three_dataset = self.boc.get_ophys_experiment_data(self.SESSION_A)
        raw_data_dct['natural_movie_three'] = movie_three_dataset.get_stimulus_template('natural_movie_three')

        natural_images_dataset = self.boc.get_ophys_experiment_data(self.SESSION_B)
        raw_data_dct['natural_scenes'] = natural_images_dataset.get_stimulus_template('natural_scenes')

        data['raw_data_dct'] = raw_data_dct
        return data

ModuleNotFoundError: No module named 'allensdk'